## Part B Q2

### Linguistic traits of the fake and real reviews
In this part, some of the linguistic and stylistic traits of the reviews are examined and the two classes are compared. The classes have been separated into fake and real classes and various aspects have been analysed. The average length of the text, the average word length, stop words, capital words, punctuation, Flesch Kinkaid reading level have been analysed. Also, counted the number of time the product title appears in the review text.


In [1]:
import sys
import nltk
# coding: utf-8
import csv  
import unicodecsv   # csv reader
import numpy as np
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline
import re
from sklearn.metrics import precision_recall_fscore_support # to report on precision and recall
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from nltk.corpus import stopwords
from string import punctuation
from textstat.textstat import textstat
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction import stop_words
import matplotlib.pyplot as plt
#import seaborn as sb
from string import ascii_lowercase, ascii_uppercase
from collections import Counter
from string import punctuation
import nltk
from nltk import bigrams
from nltk import trigrams
###Does one class contain more stopwords than the others?



In [2]:
# load data from a file and append it to the rawData
def loadData(path, Text=None):
    with open(path, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        next(reader)
        
        for line in reader:
            (Label, Text, prod_title) = parseReview(line)
            rawData.append((Label, Text, prod_title))
            preprocessedData.append((Label,preProcess(Text), prod_title))

In [3]:
def parseReview(reviewLine):
    Label = str(reviewLine[1])
    if Label == '__label1__':
        Label = 'fake'
    else:
        Label = 'real'
        
    rating = str(reviewLine[2])
    verified = str(reviewLine[3])
    prod_category = str(reviewLine[4])
    Text = str(reviewLine[8])
    prod_title = (reviewLine[6])

    return (Label, Text, prod_title)

In [4]:
def preProcess(text):
    #normalisation and tokenising 
    no_symbols = re.sub(r'[^\w]', ' ', text.lower())
    tokens = no_symbols.split()
    # Should return a list of tokens
    return tokens

In [5]:
rawData = []
preprocessedData = []
reviewPath = 'amazon_reviews.txt'
loadData(reviewPath)

In [6]:
# print(preprocessedData[:10])

In [6]:
print(rawData[:1])

[('fake', 'When least you think so, this product will save the day. Just keep it around just in case you need it for something.', 'Targus PAUK10U Ultra Mini USB Keypad, Black')]


In [7]:
fakeClass = []
realClass = []

def labelClass(text):

    for line in text:
        if line[0] == 'fake':
            fakeClass.append(line)
        else:
            realClass.append(line)
    return



In [8]:
labelClass(preprocessedData)

In [9]:
dfFakeReviews = pd.DataFrame(fakeClass)
dfRealReviews = pd.DataFrame(realClass)

In [10]:
print (fakeClass[0])

('fake', ['when', 'least', 'you', 'think', 'so', 'this', 'product', 'will', 'save', 'the', 'day', 'just', 'keep', 'it', 'around', 'just', 'in', 'case', 'you', 'need', 'it', 'for', 'something'], 'Targus PAUK10U Ultra Mini USB Keypad, Black')


In [11]:
#how long are the reviews for each class?
###On average, how long are the reviews for each class?
def AvLength(text):
    textlength = []
    for i in text[1]:
        len(i)
        textlength.append(len(i))
        
    return np.mean(textlength)

print ("The average length in fake reviews is: %d " % AvLength(dfFakeReviews))

print ("The average length in real reviews is: %d" % AvLength(dfRealReviews))

The average length in fake reviews is: 61 
The average length in real reviews is: 81


In [12]:
preData=[]
for (Label,Text, prod_title) in rawData:
    preData.append((Label,preProcess(Text),prod_title))

In [13]:
#Average word length per class
#divide fake or real
fake_words = []
real_words = []

for i in range(len(preData)):
    if (rawData[i][0] == 'fake'):
        fake_words.append(preData[i][1])
    else:
        real_words.append(preData[i][1])

In [14]:
## Counting word real and word fake
word_len_fake = []
word_len_real = []

for i in range(len(real_words)):
    for j in range(len(real_words[i])):
        word_len_real.append(len(real_words[i][j]))
        
for i in range(len(fake_words)):
    for j in range(len(fake_words[i])):
        word_len_fake.append(len(fake_words[i][j]))

In [15]:
## Average word length per class
print(np.average(word_len_real))
print(np.average(word_len_fake))

4.0762300770383675
4.057513688906916


In [16]:
#More complex measures - Flesch Kinkaid Readability Test
str_fake = []
str_real= []

fki_fake = []
fki_real = []

for i in range(len(rawData)):
    if (rawData[i][0] == 'fake'):
        str_fake.append(rawData[i][1])
    else:
        str_real.append(rawData[i][1])
        
for t in range(len(str_fake)):
    fki_fake.append(textstat.flesch_reading_ease(str_fake[t]))

for t in range(len(str_real)):
    fki_real.append(textstat.flesch_reading_ease(str_real[t]))

In [17]:
fake_average = sum(fki_fake)/len(fki_fake)
real_average = sum(fki_real)/len(fki_real)

print("The fake average is: ", fake_average)
print("The real average is: ", real_average)

The fake average is:  79.75902761904763
The real average is:  79.029706666667


In [18]:
###Does one class contain more stopwords than the others?

stopWords = set(stopwords.words('english'))
fake_stopwords = []
real_stopwords = []
real_total =[]
fake_total = []

#Total words in real label
for i in range(len(real_words)):
    for w in real_words[i]:
        real_total.append(w)
        
#Total words in fake label
for i in range(len(fake_words)):
    for w in fake_words[i]:
        fake_total.append(w)

#stopwords in real label
for i in range(len(real_words)):
    for w in real_words[i]:
        if w in stopWords:
            real_stopwords.append(w)

#stopwords in fake label
for i in range(len(fake_words)):
    for w in fake_words[i]:
        if w in stopWords:
            fake_stopwords.append(w)

print("total words in fake labels: " , len(fake_total))
print("stop words in fake labels" , len(fake_stopwords))

print("total words in real labels: " ,len(real_total))
print("stop words in real labels" , len(real_stopwords))

stopword_per_fake = len(fake_stopwords)/len(fake_total)*100

stopword_per_real = len(real_stopwords)/len(real_total)*100

print("% of stopwords fake:" , stopword_per_fake)
print("% of stopwords real:" , stopword_per_real)

total words in fake labels:  641030
stop words in fake labels 330168
total words in real labels:  857365
stop words in real labels 428835
% of stopwords fake: 51.50585776016723
% of stopwords real: 50.01778705685444


In [19]:
###The use of punctuation 

import string

count = lambda l1, l2 : len(list(filter(lambda c: c in l2, l1)))
sum_fake_punct = 0
sum_real_punct = 0

for i in range(len(str_fake)):
    for w in str_fake[i]:
        punct_fake = count(w, string.punctuation)
        sum_fake_punct += punct_fake

for i in range(len(str_real)):
    for w in str_real[i]:
        punct_real = count(w, string.punctuation)
        sum_real_punct += punct_real

In [20]:
print("punctuation in fake label:" , sum_fake_punct)
print("punctuation in real label: ", sum_real_punct)

punctuation in fake label: 106917
punctuation in real label:  163501


In [21]:
# no. of capital words in fake and real classes
fakeClassUpper = []
realClassUpper = []

def labelClass(text):

    for line in text:
        if line[0] == "fake":
            fakeClassUpper.append(line)
        else:
            realClassUpper.append(line)
    return

In [22]:
labelClass(rawData)

In [23]:
# Counting the no. of Capital words in the fake reviews
labelClass(rawData)
count=0
for record in fakeClassUpper:
    for word in record[1].split(" "):
        if word.isupper():
            count += 1   
                
print("Capital Words in real class: " + str(count))

Capital Words in real class: 53874


In [24]:
# Counting the no. of capital words in real reviews

labelClass(rawData)
count=0
for record in realClassUpper:
    for word in record[1].split(" "):
        if word.isupper():
            count += 1   
                
print("Capital Words in real class: " + str(count))

Capital Words in real class: 99039


In [6]:
# The number of times the product title appears in Reviews
tokenData=[]
for (Label,Text, prod_title) in rawData:
    tokenData.append((Label,Text, preProcess(prod_title)))
title_fake = 0
title_real= 0
for i in range(len(tokenData)):
    if (rawData[i][0] == 'fake'):
        if tokenData[i][2][0] in tokenData[i][1]:
            title_fake+=1
    else:
        if tokenData[i][2][0] in tokenData[i][1]:
            title_real+=1

In [7]:
print("Fake: " + str(title_fake))
print("Real: "+ str(title_real))

Fake: 1065
Real: 924
